In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('..')
import numpy as np
from numpy.random import random as rand

import networkqit as nq
import networkx as nx

from networkqit import graph_laplacian as GL

import scipy.optimize
import scipy.linalg

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

import sympy as sp
import sympyprinting as spp
from sympy import init_printing

import mpmath as mp

#### 1. Define the Stieltjes transform $t_r(z)$ as in Peixoto 2013, Spectra of random modular graphs and the symbols

In [ ]:
n_=sp.Symbol('n',integer=True)
pin_=sp.Symbol('pin',real=True)
pout_=sp.Symbol('pout',real=True)
c_=sp.Symbol('c',integer=True)
z_=sp.Symbol('z',complex=True)
b_=sp.Symbol('b',integer=True)
t_=sp.Symbol('t',complex=True)

# this is the average degree in the planted partition model with B blocks and n nodes
def avg_deg_planted(n,b,pin,pout):
    return pin * n / b + (b-1) * pout * n / b

def poisspdf(k,l):
    return sp.exp(-l)*(l**k)/sp.gamma(k+1)

def adj_fundamental_eq(n,b,pin,pout,t,z):
    a = 2*pin*(n-1)/(b)#avg_deg_planted(n,b,pin,pout)
    return  (t_ - sp.summation(sp.KroneckerDelta(c_,0)/(z_-c_-a*t_),(c_,0,sp.oo)))

def lapl_fundamental_eq(n,b,pin,pout,t,z):
    d = n / b*pin # average within module degree
    a = pin*n/b + (b-1)*pout*n/(b)
    return  (t_ - sp.summation(poisspdf(c_,d)/(z_-c_-a*t_),(c_,0,sp.oo)))

adj_feq = sp.lambdify((n_,b_,pin_,pout_,t_,z_),adj_fundamental_eq(n_,b_,pin_,pout_,t_,z_).replace('exp_polar','exp'), modules='mpmath')
lapl_feq = sp.lambdify((n_,b_,pin_,pout_,t_,z_),lapl_fundamental_eq(n_,b_,pin_,pout_,t_,z_).replace('exp_polar','exp'), modules='mpmath')

#### 2. Look for the spectrum of the adjacency matrix (planted partition n=50, B=2, pin=0.5,pout=0.5)

In [ ]:
n, b, pin, pout = 1000.0, 8, 1,1

In [ ]:
# Now solve the equation for t and increasing z in iterative way
t0 = (np.random.random() + np.random.random()*1j) * 1E-3
eps = 1E-9
z0 = -2*n + eps*1j
dz = 0.1
zmax = n
z = z0
t = t0

allt, allz = [], []

while np.real(z) < zmax:
    print('\r',eps,(np.real(z)/zmax)*100,'%',end='')
    t = mp.findroot(lambda x: adj_feq(n,b,pin,pout,x,z), x0 = t,solver='muller') # important to use the "muller" solver
    allz.append(z)
    z += dz
    allt.append(t)
allz = np.array([np.real(zz) for zz in allz])
allt = np.array([tt.imag for tt in allt])

#### 3. Verify the results with the corresponding numerical simulation

In [ ]:
import networkx as nx
eigs_l,eigs_a = [], []
for i in range(0,2):
    A = nx.to_numpy_array(nx.planted_partition_graph(b,int(n/b),pin,pout))
    L=np.diag(A.sum(axis=0))-A
    eigs_a.append(scipy.linalg.eigvalsh(A))
    eigs_l.append(scipy.linalg.eigvalsh(L))
eigs_a = np.array(eigs_a).flatten()
eigs_l = np.array(eigs_l).flatten()

In [ ]:
plt.figure(figsize=(32,10))
plt.hist(eigs_a,500,density=True,color='blue',alpha=0.4)
plt.plot(allz,-1/(np.pi)*allt)
plt.xlim([-100,100])
plt.show()